Introduction:

My Friend, XY, has been offered a great opportunity to work for a leading firm in Dallas city. He is very excited at this opportunity while a little bit worried about the new environment he is going to move to. Acknowledged that I have learned some geo data analysis skills, he approached me for help. So the key question is: How can my friend find a convenient and enjoyable place to live in Dallas city?
With the booming of Dallas economy and its population, this analysis is also applicable for anyone interested in exploring new opportunities in any city. The success criteria of the project will be a good recommendation of Neighborhood choice to XY based on neighborhood avenues and demographics of the neighborhoods.

Data Description:

   The data acquired for this project is a combination of data from three sources:
2.1. Scraping Dallas Neighborhood Table from website
      I first make use of Dallas neighborhoods guides page on ‘https://neighborhoods.dmagazine.com/’ to scrap the table to create a data-frame. For this, I’ve used requests and Beautifulsoup4 library to create a data-frame containing name of the 57 neighborhoods of  Dallas city with some important demographic variables including: neighborhood , population, area, population density, median age, percent of 18+ age, household income, median home value, owner percent, commute time.
2.2. Getting Coordinates of Neighborhoods by using Geocoder package : 
     When we have the names of neighborhoods, the next objective is to get the
 coordinates of these 57 major neighborhoods using geocoder class of arcgis function.
2.3. Using Foursquare Location Data:
      Foursquare data is very comprehensive and it powers location data for Apple, Uber etc. For this business problem I have used, as a part of the assignment, the Foursquare API to retrieve information about the popular spots around these 57 neighborhoods. Here I’ve chosen 100 popular venues for each neighborhood within a radius of 1 km. 
    The processing of these data will help in answering some key questions : What is the neighborhoods with parks or outside activities easily accessible? What population constititue each neighborhood? How long is the average commute time for the neighborhood? Other interesting findings of the neighborhoods from the data overall?
    The insights derived from analysis of neighborhoods will give good understanding of the avenues which help in XY’s decisions to target a new home.


In [64]:
#get packages ready to run analysis
import requests
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup

In [65]:
# to scrape data from web sources and make a dataframe
url = "https://neighborhoods.dmagazine.com/"
source = requests.get(url).text
soup = BeautifulSoup(source, 'html')
table=soup.find('table')

In [66]:
column_names = ['Neighborhood','Population','Area','Pop_density','Median_age','age18+','Household_income','Median_home_value','Owner percent','Commute_time']
df = pd.DataFrame(columns = column_names)

In [67]:
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==10:
        df.loc[len(df)] = row_data
        
df.head(2)


,Neighborhood,Population,Area,Pop_density,Median_age,age18+,Household_income,Median_home_value,Owner percent,Commute_time
0,Bent Tree,"10,215",2.59,3944.0,47.8,86.2%,"$79,127","$466,614",55.7%,23.0
1,Bluffview,"4,617",1.43,"3,228.7",38.9,75.0%,"$64,001","$624,068",61.9%,22.0


In [68]:
# the following steps are used for data cleaning so all object fields are transformed into float numbers and no missing vlaues exist in the data

df[df.columns[1:]] = df[df.columns[1:]].apply(lambda x: x.str.replace('$','')).apply(lambda x: x.str.replace(',','')).apply(lambda x: x.str.replace('%',''))
df2=df[df.columns[1:]].apply(pd.to_numeric)


In [69]:
#change the percent into decimal
df2[['age18+','Owner percent']]=df2[['age18+','Owner percent']]/100
df2.index=df.Neighborhood
df2.reset_index(inplace=True)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 10 columns):
Neighborhood         57 non-null object
Population           56 non-null float64
Area                 56 non-null float64
Pop_density          55 non-null float64
Median_age           56 non-null float64
age18+               56 non-null float64
Household_income     56 non-null float64
Median_home_value    56 non-null float64
Owner percent        56 non-null float64
Commute_time         56 non-null float64
dtypes: float64(9), object(1)
memory usage: 4.6+ KB


In [70]:
# this is the clean data of dallas neighborhood with demographics
df2.head()

,Neighborhood,Population,Area,Pop_density,Median_age,age18+,Household_income,Median_home_value,Owner percent,Commute_time
0,Bent Tree,10215.0,2.59,3944.0,47.8,0.862,79127.0,466614.0,0.557,23.0
1,Bluffview,4617.0,1.43,3228.7,38.9,0.750,64001.0,624068.0,0.619,22.0
2,Casa Linda,4175.0,1.46,2859.6,46.3,0.808,85488.0,234502.0,0.833,29.0
3,Central Dallas,148318.0,28.70,5167.9,34.4,0.834,53983.0,247887.0,0.255,24.0
4,Deep Ellum - Expo Park,2621.0,0.63,4160.3,34.0,0.879,53659.0,275131.0,0.063,23.0


In [72]:
#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

In [88]:
column_new = ['Neighborhood', 'Latitude', 'Longitude']
neighbors = pd.DataFrame(columns=column_new)
addresses=list(df2.Neighborhood+', Dallas')
addresses[0:2]

['Bent Tree, Dallas', 'Bluffview, Dallas']

In [104]:
#use the geocoder to pull the longitudal and latitude for all the neighborhoods
import sys
!{sys.executable} -m pip install geocoder

print('Packages installed.')

Packages installed.


In [105]:
import geocoder
neighborhoods=list(df2.Neighborhood)
lat_lng_coords = None

for data in range(0, len(addresses)-1):
    code = addresses[data]
    neighborhood_name = neighborhoods[data]
    
    g = geocoder.arcgis('{}, Dallas, USA'.format(code))
    lat_lng_coords = g.latlng

    neighbors = neighbors.append({ 'Neighborhood': neighborhood_name,
                                   'Latitude': lat_lng_coords[0],
                                   'Longitude': lat_lng_coords[1]}, ignore_index=True)

In [107]:
#here are the top 5 examples with lat and long
neighbors.head()

,Neighborhood,Latitude,Longitude
0,Bent Tree,32.973411,-96.826306
1,Bluffview,32.976402,-96.908401
2,Casa Linda,34.051474,-117.231948
3,Central Dallas,32.776272,-96.796856
4,Bent Tree,32.973411,-96.826306


In [141]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: ...working... done

# All requested packages already installed.



PROBLEM 2: This part is to get the lattitude and longitude of Toronto,Ontario
 and use the Geocoder package or the csv file to create a dataframe below:

In [142]:
import folium
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


In [143]:
# we will use Foursquare API to pull the top 100 venues in each dallas neighborhood
CLIENT_ID = '4DBULTOHWWTAECELRTOBDF2UT3AGPBB5Y4I2HEVVHL5EJKY3' # your Foursquare ID
CLIENT_SECRET = 'PPTVWVX1ZDLBWLQKEVFBWFISIQBK5TVJIDD1OCT2EES2GKFA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4DBULTOHWWTAECELRTOBDF2UT3AGPBB5Y4I2HEVVHL5EJKY3
CLIENT_SECRET:PPTVWVX1ZDLBWLQKEVFBWFISIQBK5TVJIDD1OCT2EES2GKFA


In [145]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=1000
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only r
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [146]:
dallas_venues = getNearbyVenues(names=neigh_da['Neighborhood'],
                                   latitudes=neigh_da['Latitude'],
                                   longitudes=neigh_da['Longitude']
                                  )

Bent Tree
Bluffview
Casa Linda
Central Dallas
Deep Ellum - Expo Park
Design District
Devonshire
Downtown
East Dallas
Eastwood
Far North Dallas
Forest Hills
Greenway Parks
Highland Park
Hillside
Hollywood Heights - Santa Monica
Kiestwood
Lake Highlands
Lake Park Estates
Lakewood
Lakewood Heights
Little Forest Hills
Lochwood
Love Field
Lower Greenville
M Streets - Vickery Place
Medical District
Midway Hollow
Mount Auburn
North Dallas
North Oak Cliff
Northaven Park
Northeast Dallas
Northwest Dallas
Northwood Hills - Valley View
Oak Cliff
Oak Lawn
Old East Dallas
Old Lake Highlands
Park Cities
Pleasant Grove
Preston Highlands
Preston Hollow
Preston Trail
Prestonwood
Ridgewood Park
South Dallas/Fair Park
Southeast Dallas
Southwest Dallas
The Peninsula
University Park
Uptown
West Dallas
Wilshire Heights
Casa View
Glen Abbey


PROBLEM 3: This part is to replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Dallas.
Explore and cluster the neighborhoods in dallas. You can decide what venues are convenientily accessible to each neighborhood.

In [147]:
dallas_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bent Tree,32.973411,-96.826306,Liberty Burger,32.969000,-96.820290,Burger Joint
1,Bent Tree,32.973411,-96.826306,Uncle Julio's,32.969441,-96.820694,Mexican Restaurant
2,Bent Tree,32.973411,-96.826306,Cavanaugh Flight Museum,32.972898,-96.834684,History Museum
3,Bent Tree,32.973411,-96.826306,Dough Bro's Italian Kitchen,32.968971,-96.820500,Italian Restaurant
4,Bent Tree,32.973411,-96.826306,Addison Airport (ADS) (Addison Airport),32.978150,-96.831028,Airport


In [148]:
#get a count of venues available in each neighborhoods within 1000meters
dallas_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bent Tree,23,23,23,23,23,23
Bluffview,41,41,41,41,41,41
Casa Linda,11,11,11,11,11,11
Casa View,36,36,36,36,36,36
Central Dallas,100,100,100,100,100,100
Deep Ellum - Expo Park,100,100,100,100,100,100
Design District,100,100,100,100,100,100
Devonshire,76,76,76,76,76,76
Downtown,100,100,100,100,100,100


In [158]:
# one hot encoding
dallas_onehot = pd.get_dummies(dallas_venues[['Venue Category']], prefix="", prefix_sep="")
dallas_onehot.insert(loc=0, column='Neighborhood', value=dallas_venues['Neighborhood'] )
dallas_onehot.shape

(2333, 247)

In [159]:
#for each neighborhood, calculate the average number of venues
dallas_grouped = dallas_onehot.groupby('Neighborhood').mean().reset_index()
dallas_grouped.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach Bar,Beer Bar,Beer Garden,Big Box Store,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Quad,College Soccer Field,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Fabric Shop,Farmers Market,Fast Food Restaurant,Fishing Spot,Flower Shop,Fondue Restaurant,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Home Service,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Office,Opera House,Optical Shop,Other Great Outdoors,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Restaurant,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Smoothie Shop,Snack Place,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train Station,Tram Station,Used Bookstore,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Bent Tree,0.0,0.00000,0.043478,0.0,0.130435,0.043478,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.00000,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00000,0.0,0.00,0.0,0.0,0.00000,0.00,0.00000,0.0,0.043478,0.0,0.000000,0.0,0.00,0.00,0.043478,0.0,0.00,0.0,0.0,0.000000,0.0,0.000000,0.00,0.043478,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.00,0.0,0.0,0.043478,0.00,0.000000,0.00,0.000000,0.0,0.043478,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.00000,0.00,0.000000,0.0,0.0

In [160]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [163]:
# return the most easily accessible venues in each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = dallas_grouped['Neighborhood']

for ind in np.arange(dallas_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dallas_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bent Tree,Airport Terminal,Spa,Intersection,History Museum,Italian Restaurant,Shipping Store,Food Truck,Burger Joint,Sandwich Place,Nail Salon
1,Bluffview,Indian Restaurant,Ice Cream Shop,Korean Restaurant,Fried Chicken Joint,Bubble Tea Shop,Burger Joint,Supermarket,Lake,Bookstore,Fast Food Restaurant
2,Casa Linda,Park,Sandwich Place,Italian Restaurant,Fast Food Restaurant,Mediterranean Restaurant,Snack Place,Dessert Shop,Grocery Store,Mexican Restaurant,Coffee Shop
3,Casa View,Mexican Restaurant,Chinese Restaurant,Thrift / Vintage Store,Pizza Place,Locksmith,Sandwich Place,Pharmacy,Bank,Dessert Shop,Business Service
4,Central Dallas,Hotel,Coffee Shop,Bar,American Restaurant,Cocktail Bar,Mexican Restaurant,Plaza,Park,Italian Restaurant,Sandwich Place


In [203]:
#clustering based on venues available
# set number of clusters
kclusters = 6

dallas_grouped_clustering = dallas_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dallas_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 4, 1, 4, 4, 4, 4, 4, 4, 4])

In [205]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

dallas_merged = neigh_da[['Neighborhood','Latitude','Longitude','Pop_density','Median_age','age18+','Household_income','Median_home_value','Owner percent','Commute_time']]

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
dallas_merged = dallas_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

dallas_merged.head()

,Neighborhood,Latitude,Longitude,Pop_density,Median_age,age18+,Household_income,Median_home_value,Owner percent,Commute_time,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bent Tree,32.973411,-96.826306,3944.0,47.8,0.862,79127.0,466614.0,0.557,23.0,4,Airport Terminal,Spa,Intersection,History Museum,Italian Restaurant,Shipping Store,Food Truck,Burger Joint,Sandwich Place,Nail Salon
1,Bluffview,32.976402,-96.908401,3228.7,38.9,0.750,64001.0,624068.0,0.619,22.0,4,Indian Restaurant,Ice Cream Shop,Korean Restaurant,Fried Chicken Joint,Bubble Tea Shop,Burger Joint,Supermarket,Lake,Bookstore,Fast Food Restaurant
2,Casa Linda,34.051474,-117.231948,2859.6,46.3,0.808,85488.0,234502.0,0.833,29.0,1,Park,Sandwich Place,Italian Restaurant,Fast Food Restaurant,Mediterranean Restaurant,Snack Place,Dessert Shop,Grocery Store,Mexican Restaurant,Coffee Shop
3,Central Dallas,32.776272,-96.796856,5167.9,34.4,0.834,53983.0,247887.0,0.255,24.0,4,Hotel,Coffee Shop,Bar,American Restaurant,Cocktail Bar,Mexican Restaurant,Plaza,Park,Italian Restaurant,Sandwich Place
4,Deep Ellum - Expo Park,32.784070,-96.782800,4160.3,34.0,0.879,53659.0,275131.0,0.063,23.0,4,Bar,Nightclub,American Restaurant,Dive Bar,Coffee Shop,Brewery,Music Venue,Burger Joint,Rock Club,Diner


In [206]:
# we can see cluster 1 has most parks available in their neighborhoods,so I would recommend cluster 1 neighborhood to my friend
dallas_merged[dallas_merged['Cluster Labels']==1]

,Neighborhood,Latitude,Longitude,Pop_density,Median_age,age18+,Household_income,Median_home_value,Owner percent,Commute_time,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Casa Linda,34.051474,-117.231948,2859.6,46.3,0.808,85488.0,234502.0,0.833,29.0,1,Park,Sandwich Place,Italian Restaurant,Fast Food Restaurant,Mediterranean Restaurant,Snack Place,Dessert Shop,Grocery Store,Mexican Restaurant,Coffee Shop
17,Lake Highlands,32.856460,-96.697950,6017.8,34.7,0.725,44539.0,266181.0,0.397,29.0,1,Park,IT Services,Pharmacy,Food,Video Store,Chinese Restaurant,Fast Food Restaurant,Pizza Place,Cosmetics Shop,Dive Bar
19,Lakewood,32.823550,-96.736310,3178.9,40.1,0.752,87462.0,436651.0,0.643,27.0,1,Park,Fishing Spot,Lake,Boat or Ferry,Trail,Golf Course,Harbor / Marina,Scenic Lookout,Dive Bar,Dog Run
20,Lakewood Heights,32.819953,-96.741775,6451.1,36.8,0.724,94521.0,408067.0,0.653,25.0,1,Locksmith,Golf Course,Park,Zoo,Dog Run,Donut Shop,Dry Cleaner,Electronics Store,Ethiopian Restaurant,Discount Store
38,Old Lake Highlands,32.856460,-96.697950,4035.8,43.3,0.785,72524.0,269270.0,0.700,27.0,1,Park,IT Services,Pharmacy,Food,Video Store,Chinese Restaurant,Fast Food Restaurant,Pizza Place,Cosmetics Shop,Dive Bar
48,Southwest Dallas,33.912940,-84.834041,1654.3,28.8,0.683,46683.0,121586.0,0.570,30.0,1,Trail,Convenience Store,Shopping Mall,Gym,Park,Light Rail Station,Fast Food Restaurant,Pizza Place,Fried Chicken Joint,Donut Shop
49,The Peninsula,32.840991,-96.710161,3191.8,45.8,0.787,81571.0,284848.0,0.819,25.0,1,Park,Bakery,Harbor / Marina,Gas Station,Used Bookstore,Italian Restaurant,Tex-Mex Restaurant,Beer Bar,Public Art,Wine Bar


In [208]:
!conda install -c conda-forge geopy --yes

Solving environment: ...working... done

# All requested packages already installed.



In [210]:
from geopy.geocoders import Nominatim

city = 'dallas, tx'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Dallas are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Dallas are 32.7762719, -96.7968559.


In [211]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=20)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dallas_merged['Latitude'], dallas_merged['Longitude'], dallas_merged['Neighborhood'], dallas_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Visualization of clusters and dallas map has validated the cluster analysis based on venues. we can see from the map that cluster 1 neighborhood most located around lakes, where is usually a reasonable sites for the parks establishment.

In [235]:
#cluster anlayis based on demographic varaibles
from sklearn.preprocessing import StandardScaler

X = neigh_da.values[:,3:]
X = np.nan_to_num(X)
cluster_dataset = StandardScaler().fit_transform(X)


In [237]:
num_clusters = 6

k_means = KMeans(init="k-means++", n_clusters=num_clusters, n_init=12)
k_means.fit(cluster_dataset)
labels = k_means.labels_

print(labels)

[5 1 1 0 2 2 3 2 0 1 0 1 5 3 1 1 1 0 1 1 1 1 1 1 1 1 2 1 0 5 0 1 0 0 1 4 2
 0 1 3 0 1 5 5 1 1 0 4 0 1 3 2 0 1 0 5]


In [241]:
neigh_da['dcluster']=labels
neigh_da.groupby('dcluster')['Pop_density','Median_age','age18+','Household_income','Median_home_value','Owner percent','Commute_time'].mean()

,Pop_density,Median_age,age18+,Household_income,Median_home_value,Owner percent,Commute_time
dcluster,,,,,,,
0,5283.071429,32.457143,0.723143,42252.714286,180224.000000,0.446071,29.000000
1,4122.937500,41.595833,0.777208,85448.541667,328356.458333,0.754583,25.791667
2,6218.800000,34.166667,0.900667,66325.833333,281634.333333,0.143833,22.166667
3,4522.600000,37.350000,0.738500,159716.250000,964580.000000,0.787000,19.000000
4,3268.350000,30.700000,0.685000,35234.500000,89608.000000,0.559000,32.500000
5,2337.216667,49.741667,0.842283,101305.833333,646573.000000,0.613850,22.333333


So, to summarize the clusters based on the demographic information:
clusters labels are aligned head to head to give my friend a better idea which clusters could be their dream neighborhood.
from least populated to most popuated:        5,4,1,3,0,2 
from youngest to eldest community:            4,0,2,3,1,5 
from highest income to lowest income:         3,5,1,2,0,4 
from highest averge houseing price to lowest: 3,5,1,2,0,4 
most renter to less renters :                 2,0,4,5,1,2  
from less commute time to most :              3,2,5,1,0,4 
based on differnt standards, the lists above ranked each neighborhood on a sinle measurement. we have to play the  trade off here to select an ideal neighborhood which is not too expansive to buy a home, which is not over populated, which has less commute time and not many people moved in and out frequently. My friend and I agree cluster 1 would be a better choice, if not best.

In [249]:
#to pull the demographic cluster 1
demo=set(neigh_da[neigh_da.dcluster==1]['Neighborhood'])

In [253]:
#to pul the venue cluster 1
venue=set(dallas_merged[dallas_merged['Cluster Labels']==1]['Neighborhood'])

In [256]:
# to list all the selected neighborhood based on demograhics
demo

{'Bluffview',
 'Casa Linda',
 'Eastwood',
 'Forest Hills',
 'Hillside',
 'Hollywood Heights - Santa Monica',
 'Kiestwood',
 'Lake Park Estates',
 'Lakewood',
 'Lakewood Heights',
 'Little Forest Hills',
 'Lochwood',
 'Love Field',
 'Lower Greenville',
 'M Streets - Vickery Place',
 'Midway Hollow',
 'Northaven Park',
 'Northwood Hills - Valley View',
 'Old Lake Highlands',
 'Preston Highlands',
 'Prestonwood',
 'Ridgewood Park',
 'The Peninsula',
 'Wilshire Heights'}

In [254]:
# to list all the selected neighborhoods based on venues,i.e. parks
venue

{'Casa Linda',
 'Lake Highlands',
 'Lakewood',
 'Lakewood Heights',
 'Old Lake Highlands',
 'Southwest Dallas',
 'The Peninsula'}

In [270]:
#  to print out the neighborhoods that meet both criterias: demograhpic and parks
print("Hi, I will recommend following neighborhood for your dream house. They are: ")
print(demo&venue)

Hi, I will recommend following neighborhood for your dream house. They are: 
{'Old Lake Highlands', 'The Peninsula', 'Lakewood', 'Lakewood Heights', 'Casa Linda'}
